# Imports

In [1]:
## Logging and Shell
import logging
logging.basicConfig(
    level=logging.ERROR,
    force=True
)

## Generic imports
from collections import defaultdict
import pandas as pd

## File I/O
from pathlib import Path
import json

# Cheminformatics
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

from openff.toolkit import Topology
from openff.toolkit.utils.exceptions import IncorrectNumConformersWarning

# Custom Imports
from polymerist.maths.greek import GREEK_PREFIXES
from polymerist.genutils.containers import RecursiveDict

from polymerist.rdutils.reactions import reactions, reactors
from polymerist.rdutils import rdkdraw

DIM    = 300
ASPECT = 3/2
rdkdraw.set_rdkdraw_size(DIM, ASPECT)

from polymerist.monomers import specification, MonomerGroup
from polymerist.residues.partition import partition
from polymerist.polymers import building

from polymerist.openfftools import topology
from polymerist.openfftools.pcharge import MolCharger

# catch annoying warnings
import warnings 
warnings.catch_warnings(record=True)
warnings.filterwarnings('ignore', category=IncorrectNumConformersWarning)

/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# Static Paths
RAW_DATA_DIR  = Path('monomer_data_raw')
FMT_DATA_DIR  = Path('monomer_data_formatted')
PROC_DATA_DIR = Path('monomer_data_processed')
RXN_FILES_DIR = Path('poly_rxns')

PDB_OUT_DIR   = Path('pdb_files')
MONO_OUT_DIR  = Path('monomer_fragments')
TOPO_OUT_DIR  = Path('Topologies')

MONO_OUT_DIR.mkdir(exist_ok=True)
PDB_OUT_DIR.mkdir(  exist_ok=True)
TOPO_OUT_DIR.mkdir( exist_ok=True)

# Load monomer and rxn data 

In [3]:
# input_data_path = PROC_DATA_DIR / '20231114_polyid_data_density_DP2-6 - 1,2 monomers_FILTERED.csv'
# input_data_path = PROC_DATA_DIR / 'nipu_urethanes_FILTERED.csv'
input_data_path = PROC_DATA_DIR / 'monomer_data_MASTER.csv'
df = pd.read_csv(input_data_path, index_col=0)

## Load pre-defined reactions with functional group and name backmap

In [4]:
keys = ['rxn_name']

blacklisted_rxns = ['imide']#, 'vinyl']
df = df[df.mechanism.map(lambda s : s not in blacklisted_rxns)]

df_grouper = df.groupby(keys)
frames = {
    mech : df_grouper.get_group(mech)
        for mech in df_grouper.groups
}

In [5]:
with (RXN_FILES_DIR / 'rxn_groups.json').open('r') as file: # load table of functional group for each reaction
    rxn_groups = json.load(file)

rxns = {
    rxnname : reactions.AnnotatedReaction.from_rxnfile(RXN_FILES_DIR / f'{rxnname}.rxn')
        for rxnname in rxn_groups.keys()
}

# Auto-generating monomer fragments and Topologies

## Set up and format progress bars to track build status

In [6]:
from time import sleep
from rich.progress import Progress
from rich.progress import (
    BarColumn,
    Progress,
    SpinnerColumn,
    TaskProgressColumn,
    TextColumn,
    TimeElapsedColumn,
    TimeRemainingColumn,
)
from rich.console import Group
from rich.live import Live

# status of individual task
status_readout = Progress(
    'STATUS:',
    TextColumn(
        '[purple]{task.fields[action]}'
    ),
    '...'
)
status_id = status_readout.add_task('[green]Current compound:', action='')

# textual display of the name of the curent polymer
compound_readout = Progress(
    TextColumn(
        'Current compound ({task.completed} / {task.total}):'
    ),
    TextColumn(
        '[blue]{task.fields[polymer_name]}',
        justify='right'
    )
)
curr_compound_id  = compound_readout.add_task('[green]Current compound:', polymer_name='')

# progress over distinct classes of mechanism
overall_progress = Progress(
    SpinnerColumn(),
    "[progress.description]{task.description}",
    BarColumn(
        
    ),
    TaskProgressColumn(),
    TextColumn(
        '({task.completed} / {task.total})'
    ),
    'At:',
    TimeElapsedColumn(),
)
curr_mechanism_id = overall_progress.add_task('[blue]Reaction mechanism', start=True, total=len(frames))

# individual progress bars within each mechanism
mechanism_progress = Progress(
    "[progress.description]{task.description}",
    BarColumn(),
    TaskProgressColumn(),
    TextColumn(
        '({task.completed} / {task.total})'
    ),
    'At:',
    TimeElapsedColumn(),
)
total_compounds = 0
mech_task_ids = {} # preprocess dataframes by mechanism to determine progress bar layout and task lengths
for rxn_name, rxn_df in frames.items():
    num_compounds = len(rxn_df)
    mech_task_ids[rxn_name] = mechanism_progress.add_task(f'[cyan]{rxn_name}', start=True, total=len(rxn_df))
    total_compounds += num_compounds
compound_readout.update(curr_compound_id, total=total_compounds)


# combine progess readouts into unified live console
group = Group(
    status_readout,
    compound_readout,
    overall_progress,
    mechanism_progress,
)

## Polymerize all SMILES into fragments, generate PDB topologies, then parameterize chemical and nonbonded info 

In [7]:
DOP = 3
charge_method = 'Espaloma-AM1-BCC'

# create directories
nmer_name = f'{GREEK_PREFIXES[DOP]}mers'
N_MER_DIR_PDB  = PDB_OUT_DIR  / nmer_name 
N_MER_DIR_TOPO = TOPO_OUT_DIR / nmer_name 

N_MER_DIR_PDB.mkdir(exist_ok=True)
N_MER_DIR_TOPO.mkdir(exist_ok=True)

# set up data structures for global output
frag_registry = RecursiveDict()
failed_pdb = RecursiveDict()
unmatched_pdb_mols = defaultdict(defaultdict)
param_cancelled = defaultdict(list)

charger = MolCharger.subclass_registry[charge_method]()
param_blacklist = [ # for some reason, these compounds' Topologies take forever to partition
    '4-N-(4-aminophenyl)-4-N-[4-(2,4,6-tritert-butylphenoxy)phenyl]benzene-1,4-diamine' 
]

# execute build loop
with Live(group, refresh_per_second=10) as live:
    # ensure bars start at 0
    for pbar in group.renderables: 
        for task_id in pbar.task_ids:
            pbar.reset(task_id)

    for rxn_name, rxn_df in frames.items():
        # look up reactive groups and pathway by rxn_name
        mech_task_id = mech_task_ids[rxn_name]
        rxn_pathway = rxns[rxn_name]
        reactor = reactors.PolymerizationReactor(rxn_pathway)
        
        # initialize output directories
        mono_dir : Path = MONO_OUT_DIR / rxn_name
        mono_dir.mkdir(exist_ok=True)

        pdb_dir : Path = N_MER_DIR_PDB / rxn_name
        pdb_dir.mkdir(exist_ok=True)

        topo_dir : Path = N_MER_DIR_TOPO / rxn_name
        topo_dir.mkdir(exist_ok=True)

        overall_progress.advance(curr_mechanism_id, advance=1)

        for (i, row) in rxn_df.iterrows():
        # 0) load reactants with IUPAC names from chemical table
            # advance progress for the current compound
            mechanism_progress.advance(mech_task_id)
            compound_readout.advance(curr_compound_id)
            
            status_readout.update(status_id, action='Gathering reactants')
            named_reactants = {}
            for j in range(2):
                reactant = Chem.MolFromSmiles(row[f'smiles_monomer_{j}'], sanitize=False)
                Chem.SanitizeMol(reactant, sanitizeOps=specification.SANITIZE_AS_KEKULE)
                named_reactants[ row[f'IUPAC_name_monomer_{j}'] ] = reactant
            initial_reactants = [reactants for reactants in named_reactants.values()] # must convert to list to pass to ChemicalReaction

            polymer_name = f'poly({"-co-".join(named_reactants.keys())})' # TODO : make sure this conforms to IUPAC standards for naming
            compound_readout.update(curr_compound_id, polymer_name=polymer_name)
            frag_registry[rxn_name][i] = polymer_name

        # 1) use rxn template to polymerize monomers into all possible fragments
            status_readout.update(status_id, action='Fragmenting via reaction mechanism')
            monogrp = MonomerGroup()
            for dimer, frags in reactor.propagate(initial_reactants):
                for assoc_group_name, rdfragment in zip(named_reactants.keys(), frags):
                    # generate spec-compliant SMARTS
                    raw_smiles = Chem.MolToSmiles(rdfragment)
                    exp_smiles = specification.expanded_SMILES(raw_smiles)
                    spec_smarts = specification.compliant_mol_SMARTS(exp_smiles)

                    # record to monomer group
                    affix = 'TERM' if MonomerGroup.is_terminal(rdfragment) else 'MID'
                    monogrp.monomers[f'{assoc_group_name}_{affix}'] = [spec_smarts]

            status_readout.update(status_id, action='Saving monomer fragments...')
            monogrp.to_file(mono_dir / f'{polymer_name}.json')

        # 2) generate PDB file from fragments
            status_readout.update(status_id, action='Generating PDB file')
            try:
                polymer = building.build_linear_polymer(monomers=monogrp, DOP=DOP, sequence='AB')  
                mol_pdb_path = pdb_dir / f'{polymer_name}.pdb'
                building.mbmol_to_openmm_pdb(mol_pdb_path, polymer)
            except Exception as e:
                failed_pdb[DOP][rxn_name][e.__class__.__name__][polymer_name] = monogrp
                continue # skip to next compounds, don't proceed with parameterization


        # 3) Parameterize topology and generate SDF 
            if any(group_name in polymer_name for group_name in param_blacklist):
                status_readout.update(status_id, action='Cancelled')
                param_cancelled[rxn_name].append(polymer_name)
                continue

            try:
                status_readout.update(status_id, action='Partitioning topology by fragments')
                offtop = Topology.from_pdb(mol_pdb_path, _custom_substructures=monogrp.monomers)
                was_partitioned = partition(offtop)
                assert(was_partitioned)

                status_readout.update(status_id, action='Assigning partial charges')
                offmol = topology.get_largest_offmol(offtop)
                offmol.name = polymer_name
                cmol = charger.charge_molecule(offmol)

                status_readout.update(status_id, action='Saving parameterized topology to SDF')
                offtop = cmol.to_topology()
                sdf_path = topo_dir / f'{polymer_name}.sdf'
                topology.topology_to_sdf(sdf_path, offtop)

            except Exception as e:
                print(f'{polymer_name} : {e}')
                unmatched_pdb_mols[str(e)][rxn_name] = polymer_name


        # Update readouts at end of process
        compound_readout.update(curr_compound_id, polymer_name='Completed!')
        sleep(0.1) # needed to give final bar enough time to catch up

# global output after processing
with (MONO_OUT_DIR / 'dataset_backmap.json').open('w') as file:
    json.dump(frag_registry, file, indent=4)

print(failed_pdb)
print(param_cancelled)

Output()

/home/timber/miniconda3/envs/polymerist-env/lib/python3.11/site-packages/dgl/heterograph.py:92: DGLWarning: 
Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(

[20:41:27] Explicit valence for atom # 19 N, 4, is greater than permitted

RecursiveDict(<class 'polymerist.genutils.containers.RecursiveDict'>,
              {3: RecursiveDict(<class 'polymerist.genutils.containers.RecursiveDict'>,
                                {'polyurethane_isocyanate': RecursiveDict(<class 'polymerist.genutils.containers.RecursiveDict'>,
                                                                          {'AtomValenceException': RecursiveDict(<class 'polymerist.genutils.containers.RecursiveDict'>,
                                                                                                                 {'poly(1-isocyanato-4-[(4-isocyanatophenyl)methyl]benzene-co-2-[2-hydroxyethyl-[4-(4-nitrophenyl)diazenylphenyl]amino]ethanol)': MonomerGroup(monomers={'1-isocyanato-4-[(4-isocyanatophenyl)methyl]benzene_MID': ['[4*:1]-[#6D3+0:2](=[#8D1+0:3])-[#7D3+0:4](-[#6D3+0:5]1=[#6D3+0:6](-[#1D1+0:23])-[#6D3+0:7](-[#1D1+0:24])=[#6D3+0:8](-[#6D4+0:9](-[#6D3+0:10]2=[#6D3+0:11](-[#1D1+0:27])-[#6D3+0:12](-[#1D1+0:28])=[#6D3+0:13](-[#7D3+0:14]